In [1]:
import subprocess
import os
import csv
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [2]:
# function to print elapsed time
def calculate_elapsed_time(start_time, end_time):
    elapsed_time = end_time - start_time
    total_seconds = elapsed_time.total_seconds()
    seconds = int(total_seconds % 60)
    total_minutes = total_seconds // 60
    minutes = int(total_minutes % 60)
    hours = int(total_minutes // 60)
    
    seconds = str(seconds).zfill(2)
    minutes = str(minutes).zfill(2)
    hours = str(hours).zfill(2)

    return str(hours) + ':' + str(minutes) + ':' + str(seconds)

# Initialise

### Read parameter sets to simulate

In [3]:
table = pd.read_csv('data/parametersets_restricted.csv')
table

,alphaWalk,alphaBike,alphaCarDriver,alphaCarPassenger,alphaBus,alphaTrain,betaTimeWalk,betaTimeBike,betaTimeCarDriver,betaTimeCarPassenger,betaTimeBus,betaTimeTrain,betaCostCarDriver,betaCostCarPassenger,betaCostBus,betaCostTrain,betaTimeWalkTransport,betaChangesTransport,used_parameter_label,used_parameter_value
0,-10.0,0.0,0.0,0.0,0.0,0.0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaWalk,-10.0
1,-8.0,0.0,0.0,0.0,0.0,0.0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaWalk,-8.0
2,-6.0,0.0,0.0,0.0,0.0,0.0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaWalk,-6.0
3,-4.0,0.0,0.0,0.0,0.0,0.0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaWalk,-4.0
4,-2.0,0.0,0.0,0.0,0.0,0.0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaWalk,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,0.0,0.0,0.0,0.0,0.0,2.0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaTrain,2.0
62,0.0,0.0,0.0,0.0,0.0,4.0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaTrain,4.0
63,0.0,0.0,0.0,0.0,0.0,6.0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaTrain,6.0
64,0.0,0.0,0.0,0.0,0.0,8.0,-0.04,-0.03,-0.02,-0.02,-0.02,-0.02,-0.15,-0.15,-0.1,-0.1,-0.03,-0.3,alphaTrain,8.0


### Initialise empty the output dataframe

In [4]:
df = pd.DataFrame(columns=["CAR_DRIVER", "CAR_PASSENGER", "BIKE", "BUS_TRAM", "TRAIN", "WALK"])

for i in range(0, len(table)):
    df.to_csv("output/iterations/output_proportions_" + str(i) + ".csv", index=False)

# Call Sim2APL iteratively

### Prepare the Java command

In [5]:
# baseline of the Java command
config = "--config src/main/resources/config_DHZW_full.toml"
modeliberal = "--mode-liberal 0.5"
modeconservative = "--mode-conservative 0.5"
output = "-o output/deskrun"
parametersPath = "--parameter_file ../data/parametersets_restricted.csv"

base_cmd = "java -cp target/sim2apl-dhzw-simulation-1.0-SNAPSHOT-jar-with-dependencies.jar main.java.nl.uu.iss.ga.Simulation" + " " + config + " " + modeliberal + " " + modeconservative + " " + output + " " + parametersPath

# set current directory the folder of Sim2APL so I can execute the jar with the correct paths
if(os.path.basename(os.getcwd()) != 'DHZW-simulation_Sim-2APL'):
    new_directory = os.path.join(os.getcwd(), 'DHZW-simulation_Sim-2APL')
    new_directory = new_directory.replace('\\', '/')
    os.chdir(new_directory)

### Main loop to call the simulations

In [6]:
idx_restart = 0

In [7]:
time_beginning_all = datetime.now()

for idx, row in tqdm(table.iterrows(), total=table.shape[0]):
    if idx >= idx_restart:
        # repeat each parameter set 5 times
        for iteration in range(0, 5):

            # output file for this iteration
            outputPath = f'--output_file=../output/iterations/output_proportions_{idx}.csv'
            cmd = base_cmd + ' ' + outputPath
            
            # parameterset to use
            arg = f'--parameterset_index={idx+1}'
            full_command = f'{cmd} {arg}'
            
            time_beginning_iteration = datetime.now()
            
            print('parameter number: ' + str(idx) + ' - iteration: ' + str(iteration) + ' START || Time now: ' + time_beginning_iteration.strftime("%H:%M:%S"))
                                    
            # Capture the output of the Java program
            try:
                output = subprocess.check_output(full_command, stderr=subprocess.STDOUT, universal_newlines=True)
            except subprocess.CalledProcessError as e:
                print(f"Java program exited with non-zero return code: {e.returncode}")
                print(f"Error message: {e.output}")
                exit(1)
                
            time_end_iteration = datetime.now()
         
            print('parameter number: ' + str(idx) + ' - iteration: ' + str(iteration) + ' END   || Time now: ' + time_end_iteration.strftime("%H:%M:%S") + ' | Duration iteration: ' + calculate_elapsed_time(time_beginning_iteration, time_end_iteration) + ' | Elapsed from start: ' + calculate_elapsed_time(time_beginning_all, time_end_iteration))

  0%|                                                                                           | 0/66 [00:00<?, ?it/s]

parameter number: 0 - iteration: 0 START || Time now: 14:08:06
parameter number: 0 - iteration: 0 END   || Time now: 14:08:31 | Duration iteration: 00:00:24 | Elapsed from start: 00:00:24
parameter number: 0 - iteration: 1 START || Time now: 14:08:31
parameter number: 0 - iteration: 1 END   || Time now: 14:08:56 | Duration iteration: 00:00:24 | Elapsed from start: 00:00:49
parameter number: 0 - iteration: 2 START || Time now: 14:08:56
parameter number: 0 - iteration: 2 END   || Time now: 14:09:21 | Duration iteration: 00:00:24 | Elapsed from start: 00:01:14
parameter number: 0 - iteration: 3 START || Time now: 14:09:21
parameter number: 0 - iteration: 3 END   || Time now: 14:09:44 | Duration iteration: 00:00:23 | Elapsed from start: 00:01:38
parameter number: 0 - iteration: 4 START || Time now: 14:09:44


  2%|█▏                                                                              | 1/66 [02:01<2:11:26, 121.34s/it]

parameter number: 0 - iteration: 4 END   || Time now: 14:10:08 | Duration iteration: 00:00:23 | Elapsed from start: 00:02:01
parameter number: 1 - iteration: 0 START || Time now: 14:10:08
parameter number: 1 - iteration: 0 END   || Time now: 14:10:31 | Duration iteration: 00:00:23 | Elapsed from start: 00:02:24
parameter number: 1 - iteration: 1 START || Time now: 14:10:31
parameter number: 1 - iteration: 1 END   || Time now: 14:10:56 | Duration iteration: 00:00:25 | Elapsed from start: 00:02:49
parameter number: 1 - iteration: 2 START || Time now: 14:10:56
parameter number: 1 - iteration: 2 END   || Time now: 14:11:19 | Duration iteration: 00:00:23 | Elapsed from start: 00:03:13
parameter number: 1 - iteration: 3 START || Time now: 14:11:19
parameter number: 1 - iteration: 3 END   || Time now: 14:11:42 | Duration iteration: 00:00:23 | Elapsed from start: 00:03:36
parameter number: 1 - iteration: 4 START || Time now: 14:11:42


  3%|██▍                                                                             | 2/66 [03:59<2:07:20, 119.38s/it]

...

parameter number: 63 - iteration: 4 END   || Time now: 16:14:15 | Duration iteration: 00:00:24 | Elapsed from start: 02:06:09
parameter number: 64 - iteration: 0 START || Time now: 16:14:15
parameter number: 64 - iteration: 0 END   || Time now: 16:14:40 | Duration iteration: 00:00:24 | Elapsed from start: 02:06:33
parameter number: 64 - iteration: 1 START || Time now: 16:14:40
parameter number: 64 - iteration: 1 END   || Time now: 16:15:04 | Duration iteration: 00:00:24 | Elapsed from start: 02:06:58
parameter number: 64 - iteration: 2 START || Time now: 16:15:04
parameter number: 64 - iteration: 2 END   || Time now: 16:15:31 | Duration iteration: 00:00:26 | Elapsed from start: 02:07:24
parameter number: 64 - iteration: 3 START || Time now: 16:15:31
parameter number: 64 - iteration: 3 END   || Time now: 16:15:55 | Duration iteration: 00:00:24 | Elapsed from start: 02:07:49
parameter number: 64 - iteration: 4 START || Time now: 16:15:55


 98%|█████████████████████████████████████████████████████████████████████████████▊ | 65/66 [2:08:13<01:59, 119.82s/it]

parameter number: 64 - iteration: 4 END   || Time now: 16:16:20 | Duration iteration: 00:00:24 | Elapsed from start: 02:08:13
parameter number: 65 - iteration: 0 START || Time now: 16:16:20
parameter number: 65 - iteration: 0 END   || Time now: 16:16:44 | Duration iteration: 00:00:24 | Elapsed from start: 02:08:37
parameter number: 65 - iteration: 1 START || Time now: 16:16:44
parameter number: 65 - iteration: 1 END   || Time now: 16:17:09 | Duration iteration: 00:00:24 | Elapsed from start: 02:09:02
parameter number: 65 - iteration: 2 START || Time now: 16:17:09
parameter number: 65 - iteration: 2 END   || Time now: 16:17:32 | Duration iteration: 00:00:23 | Elapsed from start: 02:09:26
parameter number: 65 - iteration: 3 START || Time now: 16:17:32
parameter number: 65 - iteration: 3 END   || Time now: 16:17:55 | Duration iteration: 00:00:22 | Elapsed from start: 02:09:48
parameter number: 65 - iteration: 4 START || Time now: 16:17:55


100%|███████████████████████████████████████████████████████████████████████████████| 66/66 [2:10:12<00:00, 118.37s/it]

parameter number: 65 - iteration: 4 END   || Time now: 16:18:18 | Duration iteration: 00:00:23 | Elapsed from start: 02:10:12
